In [1]:
import os
import numpy as np
import open3d as o3d
import pyvista as pv


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
sourceMeshPath = "assets/library/smile_arch_origin.stl"
targetMeshPath = "results/partial_smile_arch.stl"

In [3]:
source = o3d.io.read_triangle_mesh(sourceMeshPath)
target = o3d.io.read_triangle_mesh(targetMeshPath)

source.compute_vertex_normals()
target.compute_vertex_normals()

TriangleMesh with 40103 points and 13369 triangles.

In [4]:
def o3d_to_pyvista(mesh):
    vertices = np.asarray(mesh.vertices)
    faces = np.asarray(mesh.triangles)
    
    # PyVista expects faces with leading length indicators
    faces_with_count = np.column_stack((np.full(len(faces), 3), faces))
    faces_with_count = faces_with_count.flatten()
    
    # Create PyVista PolyData
    pv_mesh = pv.PolyData(vertices, faces_with_count)
    return pv_mesh

In [5]:
source_pv = o3d_to_pyvista(source)
target_pv = o3d_to_pyvista(target)

# 5. PyVista로 시각화
plotter = pv.Plotter()

# Add source mesh in blue
plotter.add_mesh(source_pv, color='lightblue', opacity=0.7, label='Source Mesh')

# Add target mesh in green
plotter.add_mesh(target_pv, color='lightgreen', opacity=0.7, label='Target Mesh')

# Add legend
plotter.add_legend()

# Show the plot
plotter.show()

Widget(value='<iframe src="http://localhost:57339/index.html?ui=P_0x225fcbc6740_0&reconnect=auto" class="pyvis…

In [6]:
def o3d_pcd_to_pyvista(pcd):
    points = np.asarray(pcd.points)
    pv_mesh = pv.PolyData(points)  # points만 있는 PolyData 생성
    return pv_mesh

In [10]:
import open3d as o3d
import pyvista as pv
import numpy as np

def o3d_pcd_to_pyvista(pcd):
    points = np.asarray(pcd.points)
    print(f"Converting points to PyVista. Points shape: {points.shape}")
    pv_mesh = pv.PolyData(points)
    return pv_mesh

pcd_source = source.sample_points_uniformly(number_of_points=20000)
pcd_target = target.sample_points_uniformly(number_of_points=20000)

# 6. 시각화
print("\n6. Visualizing...")
plotter = pv.Plotter()
plotter.background_color = 'black'


# 소스 포인트 클라우드
source_pv = o3d_pcd_to_pyvista(pcd_source)
plotter.add_mesh(source_pv, 
                color='yellow',
                opacity=1.0,
                label='Source Points',
                render_points_as_spheres=True,
                point_size=5)  # 점 크기 줄임

# 타겟 포인트 클라우드
target_pv = o3d_pcd_to_pyvista(pcd_target)
plotter.add_mesh(target_pv, 
                color='cyan',
                opacity=1.0,
                label='Target Points',
                render_points_as_spheres=True,
                point_size=5)  # 점 크기 줄임

# 카메라 위치 설정
plotter.camera_position = 'xy'  # xy 평면에서 보기
plotter.camera.zoom(1.5)  # 확대

# 축 추가
plotter.add_axes()  # 좌표축 표시
plotter.add_bounding_box()  # 경계 상자 추가

print("\nAdding meshes to plotter completed.")
plotter.add_legend()
print("Showing plot...")
plotter.show()


6. Visualizing...
Converting points to PyVista. Points shape: (20000, 3)
Converting points to PyVista. Points shape: (20000, 3)

Adding meshes to plotter completed.
Showing plot...


Widget(value='<iframe src="http://localhost:57339/index.html?ui=P_0x225a19cbb50_3&reconnect=auto" class="pyvis…

In [11]:
o3d.visualization.draw_geometries([pcd_source, pcd_target])


In [12]:
import open3d as o3d
import numpy as np

# ... (이전 포인트 클라우드 처리 코드는 동일) ...

# Open3D 시각화
print("\n6. Visualizing with Open3D...")

# 포인트 클라우드 색상 설정
pcd_source.paint_uniform_color([1, 0.706, 0])  # 노란색
pcd_target.paint_uniform_color([0, 0.651, 0.929])  # 하늘색

# 시각화
vis = o3d.visualization.Visualizer()
vis.create_window()

# 포인트 클라우드 추가
vis.add_geometry(pcd_source)
vis.add_geometry(pcd_target)

# 렌더링 옵션 설정
opt = vis.get_render_option()
opt.background_color = np.asarray([0, 0, 0])  # 검은 배경
opt.point_size = 3.0  # 점 크기 설정

# 초기 카메라 뷰포인트 설정
ctr = vis.get_view_control()
ctr.set_zoom(0.8)

# 시각화 실행
vis.run()
vis.destroy_window()


6. Visualizing with Open3D...


In [ ]:
import open3d as o3d
import numpy as np
import copy

# 1. 메쉬 로드
print("1. Loading meshes...")
source = o3d.io.read_triangle_mesh(sourceMeshPath)
target = o3d.io.read_triangle_mesh(targetMeshPath)

# 각 단계별로 확인
print(f"Source mesh: {source}")
print(f"Target mesh: {target}")

# 2. 메쉬를 포인트 클라우드로 변환
print("\n2. Converting to point clouds...")
source_pcd = source.sample_points_uniformly(number_of_points=5000)
target_pcd = target.sample_points_uniformly(number_of_points=5000)

print(f"Source point cloud: {source_pcd}")
print(f"Target point cloud: {target_pcd}")

# 먼저 각각의 포인트 클라우드를 따로 시각화해보기
print("\nViewing source point cloud alone...")
source_pcd.paint_uniform_color([1, 0, 0])  # 빨간색
o3d.visualization.draw_geometries([source_pcd])

print("\nViewing target point cloud alone...")
target_pcd.paint_uniform_color([0, 1, 0])  # 초록색
o3d.visualization.draw_geometries([target_pcd])

# 포인트 클라우드의 바운딩 박스 확인
print("\nPoint cloud bounds:")
print("Source bounds:", source_pcd.get_axis_aligned_bounding_box())
print("Target bounds:", target_pcd.get_axis_aligned_bounding_box())

# 두 포인트 클라우드 함께 시각화
print("\nViewing both point clouds together...")
o3d.visualization.draw_geometries([source_pcd, target_pcd])

In [13]:
import open3d as o3d
import numpy as np
import copy

# 1. 메쉬 로드 및 포인트 클라우드 변환
source = o3d.io.read_triangle_mesh(sourceMeshPath)
target = o3d.io.read_triangle_mesh(targetMeshPath)

# 더 많은 포인트로 샘플링
source_pcd = source.sample_points_uniformly(number_of_points=20000)
target_pcd = target.sample_points_uniformly(number_of_points=20000)

# 2. Y=0 평면을 기준으로 정렬 (교합면 정렬)
def align_to_occlusal_plane(pcd):
    points = np.asarray(pcd.points)
    y_max = np.max(points[:, 1])
    pcd.translate([0, -y_max, 0])  # Y 최대값을 0으로 이동
    return pcd

source_pcd = align_to_occlusal_plane(source_pcd)
target_pcd = align_to_occlusal_plane(target_pcd)

# 3. Voxel 다운샘플링
voxel_size = 0.3  # 메쉬 크기를 고려한 voxel 크기
source_down = source_pcd.voxel_down_sample(voxel_size=voxel_size)
target_down = target_pcd.voxel_down_sample(voxel_size=voxel_size)

# 4. 법선 벡터 계산
source_down.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))
target_down.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))

# 5. ICP 정합 시도 (여러 초기 위치에서)
def try_icp_registration(source, target, init_transform):
    threshold = voxel_size
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, init_transform,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
    return reg_p2p

# Y축 중심 회전 시도
best_result = None
best_fitness = 0

for y_angle in np.linspace(0, 2*np.pi, 8):  # 8개의 다른 각도에서 시도
    # Y축 회전 행렬
    init_transform = np.array([
        [np.cos(y_angle), 0, np.sin(y_angle), 0],
        [0, 1, 0, 0],
        [-np.sin(y_angle), 0, np.cos(y_angle), 0],
        [0, 0, 0, 1]
    ])
    
    result = try_icp_registration(source_down, target_down, init_transform)
    print(f"Angle: {y_angle:.2f}, Fitness: {result.fitness:.4f}")
    
    if result.fitness > best_fitness:
        best_fitness = result.fitness
        best_result = result

print("\nBest registration result:")
print(best_result)

# 6. 결과 시각화
source_pcd.paint_uniform_color([1, 0, 0])      # 빨간색
target_pcd.paint_uniform_color([0, 1, 0])      # 초록색

# 변환 적용
source_transformed = copy.deepcopy(source_pcd)
source_transformed.transform(best_result.transformation)
source_transformed.paint_uniform_color([0, 0, 1])  # 파란색

print("\nViewing registration result...")
o3d.visualization.draw_geometries([source_transformed, target_pcd])

Angle: 0.00, Fitness: 0.2263
Angle: 0.90, Fitness: 0.1936
Angle: 1.80, Fitness: 0.1638
Angle: 2.69, Fitness: 0.0513
Angle: 3.59, Fitness: 0.0379
Angle: 4.49, Fitness: 0.1586
Angle: 5.39, Fitness: 0.1367
Angle: 6.28, Fitness: 0.2263

Best registration result:
RegistrationResult with fitness=2.263016e-01, inlier_rmse=1.953389e-01, and correspondence_set size of 1843
Access transformation to get result.

Viewing registration result...


In [ ]:
import open3d as o3d
import numpy as np
import copy

# 1. 메쉬 로드 및 포인트 클라우드 변환
source = o3d.io.read_triangle_mesh(sourceMeshPath)
target = o3d.io.read_triangle_mesh(targetMeshPath)

source_pcd = source.sample_points_uniformly(number_of_points=20000)
target_pcd = target.sample_points_uniformly(number_of_points=20000)

# 2. Y=0 평면 정렬
def align_to_occlusal_plane(pcd):
    points = np.asarray(pcd.points)
    y_max = np.max(points[:, 1])
    pcd.translate([0, -y_max, 0])
    return pcd

source_pcd = align_to_occlusal_plane(source_pcd)
target_pcd = align_to_occlusal_plane(target_pcd)

# 3. Voxel 다운샘플링
voxel_size = 0.3
source_down = source_pcd.voxel_down_sample(voxel_size=voxel_size)
target_down = target_pcd.voxel_down_sample(voxel_size=voxel_size)

# 4. 법선 벡터 계산
source_down.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))
target_down.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))

# 5. ICP 정합 시도 (여러 위치에서)
def try_icp_registration(source, target, init_transform):
    threshold = voxel_size
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, init_transform,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000))
    return reg_p2p

# X축 이동 범위 설정 (-5mm에서 +5mm까지)
x_translations = np.linspace(-5, 5, 5)  # -5mm에서 5mm까지 5개 지점
x_rotations = np.linspace(-np.pi/8, np.pi/8, 5)  # -45도에서 45도까지 5개 지점
y_rotations = np.linspace(-np.pi/8, np.pi/8, 5)  # -45도에서 45도까지 5개 지점
z_rotations = np.linspace(-np.pi/8, np.pi/8, 5)  # -45도에서 45도까지 5개 지점
best_result = None
best_fitness = 0

print("Trying different positions and rotations...")
for x_trans in x_translations:
    for x_angle in x_rotations:
        for y_angle in y_rotations:
            for z_angle in z_rotations:
                # X, Y, Z축 회전 + X축 이동 변환 행렬
                Rx = np.array([
                    [1, 0, 0],
                    [0, np.cos(x_angle), -np.sin(x_angle)],
                    [0, np.sin(x_angle), np.cos(x_angle)]
                ])
                Ry = np.array([
                    [np.cos(y_angle), 0, np.sin(y_angle)],
                    [0, 1, 0],
                    [-np.sin(y_angle), 0, np.cos(y_angle)]
                ])
                Rz = np.array([
                    [np.cos(z_angle), -np.sin(z_angle), 0],
                    [np.sin(z_angle), np.cos(z_angle), 0],
                    [0, 0, 1]
                ])
                
                # 회전 행렬 결합
                R = Rz @ Ry @ Rx
                
                init_transform = np.eye(4)
                init_transform[:3, :3] = R
                init_transform[0, 3] = x_trans
                
                result = try_icp_registration(source_down, target_down, init_transform)
                #print(f"X translation: {x_trans:.1f}, Angles: x={np.degrees(x_angle):.1f}, y={np.degrees(y_angle):.1f}, z={np.degrees(z_angle):.1f}, Fitness: {result.fitness:.4f}")
                
                if result.fitness > best_fitness:
                    best_fitness = result.fitness
                    best_result = result

print("\nBest registration result:")
print(best_result)
print(f"Best fitness: {best_fitness}")

# 6. 결과 시각화
source_pcd.paint_uniform_color([1, 0, 0])      # 빨간색 (원본)
target_pcd.paint_uniform_color([0, 1, 0])      # 초록색 (타겟)

# 최적의 변환 적용
source_transformed = copy.deepcopy(source_pcd)
source_transformed.transform(best_result.transformation)
source_transformed.paint_uniform_color([0, 0, 1])  # 파란색 (변환됨)

print("\nViewing original state...")
o3d.visualization.draw_geometries([source_pcd, target_pcd])

print("\nViewing registration result...")
o3d.visualization.draw_geometries([source_transformed, target_pcd])

# 최적의 변환 행렬 출력
print("\nBest transformation matrix:")
print(np.array2string(best_result.transformation, precision=6, suppress_small=True))

# 변환 행렬 분석
rotation = best_result.transformation[:3, :3]
translation = best_result.transformation[:3, 3]

# 회전 각도 계산 (Y축 회전)
angle_y = np.arctan2(rotation[0, 2], rotation[0, 0])
angle_degrees = np.degrees(angle_y)

print("\nTransformation analysis:")
print(f"Translation vector [x, y, z]: {translation}")
print(f"Rotation angle around Y-axis: {angle_degrees:.2f} degrees")

# 정합 품질 메트릭
print("\nRegistration quality metrics:")
print(f"Fitness: {best_result.fitness}")
print(f"Inlier RMSE: {best_result.inlier_rmse}")

In [14]:
import open3d as o3d
import numpy as np
import copy

# 주축 정렬까지는 동일
print("1. Initial principal axis alignment...")
source_mean, source_axes = get_principal_axes(source_pcd)
target_mean, target_axes = get_principal_axes(target_pcd)

R = target_axes @ source_axes.T
t = target_mean - R @ source_mean

init_transform = np.eye(4)
init_transform[:3, :3] = R
init_transform[:3, 3] = t

# 미세 조정을 위한 추가 변환 시도
print("\n2. Fine-tuning with multiple positions...")
x_translations = np.linspace(-2, 2, 5)  # ±2mm
x_rotations = np.linspace(-np.pi/6, np.pi/6, 5)  # ±30도
y_rotations = np.linspace(-np.pi/6, np.pi/6, 5)  # ±30도
z_rotations = np.linspace(-np.pi/6, np.pi/6, 5)  # ±30도

best_result = None
best_fitness = 0

for x_trans in x_translations:
    for x_angle in x_rotations:
        for y_angle in y_rotations:
            for z_angle in z_rotations:
                # X축 회전 행렬
                Rx = np.array([
                    [1, 0, 0],
                    [0, np.cos(x_angle), -np.sin(x_angle)],
                    [0, np.sin(x_angle), np.cos(x_angle)]
                ])
                
                # Y축 회전 행렬
                Ry = np.array([
                    [np.cos(y_angle), 0, np.sin(y_angle)],
                    [0, 1, 0],
                    [-np.sin(y_angle), 0, np.cos(y_angle)]
                ])
                
                # Z축 회전 행렬
                Rz = np.array([
                    [np.cos(z_angle), -np.sin(z_angle), 0],
                    [np.sin(z_angle), np.cos(z_angle), 0],
                    [0, 0, 1]
                ])
                
                # 전체 회전 행렬 (Z -> Y -> X 순서로 적용)
                R_fine = Rx @ Ry @ Rz
                
                # 미세 조정 변환 행렬
                fine_transform = np.eye(4)
                fine_transform[:3, :3] = R_fine
                fine_transform[0, 3] = x_trans
                
                # 초기 정렬에 미세 조정을 결합
                combined_transform = fine_transform @ init_transform
                
                result = o3d.pipelines.registration.registration_icp(
                    source_pcd, target_pcd,
                    0.5,  # threshold
                    combined_transform,
                    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100)
                )
                
                print(f"X_trans: {x_trans:.1f}, X_rot: {np.degrees(x_angle):.1f}°, Y_rot: {np.degrees(y_angle):.1f}°, Z_rot: {np.degrees(z_angle):.1f}°, Fitness: {result.fitness:.4f}")
                
                if result.fitness > best_fitness:
                    best_fitness = result.fitness
                    best_result = result

print("\nBest registration result:")
print(best_result)
print(f"\nBest fitness: {best_fitness}")

# 결과 시각화
source_pcd.paint_uniform_color([1, 0, 0])      # 빨간색
target_pcd.paint_uniform_color([0, 1, 0])      # 초록색

source_transformed = copy.deepcopy(source_pcd)
source_transformed.transform(best_result.transformation)
source_transformed.paint_uniform_color([0, 0, 1])  # 파란색

# 좌표축 추가
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=5.0)

print("\nViewing final result...")
o3d.visualization.draw_geometries([source_transformed, target_pcd, coordinate_frame])

1. Initial principal axis alignment...


NameError: name 'get_principal_axes' is not defined

In [18]:
import open3d as o3d
import numpy as np
import copy

def draw_registration_result(source, target, transformation=None):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0, 0])
    target_temp.paint_uniform_color([0, 1, 0])
    
    if transformation is not None:
        source_temp.transform(transformation)
    
    coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=5.0)
    
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(source_temp)
    vis.add_geometry(target_temp)
    vis.add_geometry(coord)
    
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])
    opt.point_size = 2.0
    
    vis.run()
    vis.destroy_window()

def prepare_point_cloud(pcd, voxel_size):
    pcd_down = pcd.voxel_down_sample(voxel_size)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))
    return pcd_down

def fast_registration(source, target):
    source_points = np.asarray(source.points)
    target_points = np.asarray(target.points)
    
    source_mean = np.mean(source_points, axis=0)
    target_mean = np.mean(target_points, axis=0)
    
    init_translation = target_mean - source_mean
    
    # 각 축별 회전 각도 (-15도에서 +15도, 7.5도 간격)
    angles = np.linspace(-np.pi/12, np.pi/12, 5)  # [-15°, -7.5°, 0°, 7.5°, 15°]
    
    # X축 이동 범위
    x_offsets = list(range(-10, 11, 5))
    
    best_fitness = 0
    best_transform = None
    total_tries = len(angles) * len(angles) * len(angles) * len(x_offsets)
    current_try = 0
    
    estimation = o3d.pipelines.registration.TransformationEstimationPointToPlane()
    criteria = o3d.pipelines.registration.ICPConvergenceCriteria(
        relative_fitness=1e-6,
        relative_rmse=1e-6,
        max_iteration=30
    )
    
    # 각 축별 회전 행렬 생성 함수
    def create_rotation_matrix(angle_x, angle_y, angle_z):
        # X축 회전
        Rx = np.array([
            [1, 0, 0],
            [0, np.cos(angle_x), -np.sin(angle_x)],
            [0, np.sin(angle_x), np.cos(angle_x)]
        ])
        
        # Y축 회전
        Ry = np.array([
            [np.cos(angle_y), 0, np.sin(angle_y)],
            [0, 1, 0],
            [-np.sin(angle_y), 0, np.cos(angle_y)]
        ])
        
        # Z축 회전
        Rz = np.array([
            [np.cos(angle_z), -np.sin(angle_z), 0],
            [np.sin(angle_z), np.cos(angle_z), 0],
            [0, 0, 1]
        ])
        
        # 회전 순서: X -> Y -> Z
        return Rz @ Ry @ Rx
    
    for angle_x in angles:  # X축 회전 (빨간색 축)
        for angle_y in angles:  # Y축 회전 (초록색 축)
            for angle_z in angles:  # Z축 회전 (파란색 축)
                for x_offset in x_offsets:
                    current_try += 1
                    print(f"\rTrying combination {current_try}/{total_tries} "
                          f"(X: {np.degrees(angle_x):.1f}°, "
                          f"Y: {np.degrees(angle_y):.1f}°, "
                          f"Z: {np.degrees(angle_z):.1f}°, "
                          f"offset: {x_offset}mm)", end="")
                    
                    # 회전 행렬 생성
                    R = create_rotation_matrix(angle_x, angle_y, angle_z)
                    
                    # 변환 행렬 생성
                    transform = np.eye(4)
                    transform[:3, :3] = R
                    transform[:3, 3] = init_translation + np.array([x_offset, 0, 0])
                    
                    # Trimmed ICP 실행
                    result = o3d.pipelines.registration.registration_icp(
                        source, target,
                        max_correspondence_distance=1.0,
                        init=transform,
                        estimation_method=estimation,
                        criteria=criteria
                    )
                    
                    if result.fitness > best_fitness:
                        best_fitness = result.fitness
                        best_transform = result.transformation
                        print(f"\nNew best fitness: {best_fitness:.6f} "
                              f"(X: {np.degrees(angle_x):.1f}°, "
                              f"Y: {np.degrees(angle_y):.1f}°, "
                              f"Z: {np.degrees(angle_z):.1f}°, "
                              f"offset: {x_offset}mm)")
    
    print("\n")
    return best_transform, best_fitness

# 메인 코드는 이전과 동일
print("1. Loading point clouds...")
source = o3d.io.read_triangle_mesh(sourceMeshPath)
target = o3d.io.read_triangle_mesh(targetMeshPath)

voxel_size = 0.5
source_pcd = source.sample_points_uniformly(number_of_points=50000)
target_pcd = target.sample_points_uniformly(number_of_points=50000)

print("2. Preparing point clouds...")
source_down = prepare_point_cloud(source_pcd, voxel_size)
target_down = prepare_point_cloud(target_pcd, voxel_size)

print("3. Running fast registration...")
best_transform, initial_fitness = fast_registration(source_down, target_down)

print(f"Initial alignment complete.")
print(f"Initial fitness: {initial_fitness}")

print("\n4. Final refinement...")
final_result = o3d.pipelines.registration.registration_icp(
    source_pcd, target_pcd,
    max_correspondence_distance=voxel_size * 0.4,
    init=best_transform,
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane(),
    criteria=o3d.pipelines.registration.ICPConvergenceCriteria(
        relative_fitness=1e-6,
        relative_rmse=1e-6,
        max_iteration=50
    )
)

print(f"Final fitness: {final_result.fitness}")
print(f"Final RMSE: {final_result.inlier_rmse}")

print("\nViewing final result...")
draw_registration_result(source_pcd, target_pcd, final_result.transformation)

print("\nFinal transformation matrix:")
print(np.array2string(final_result.transformation, precision=6, suppress_small=True))

1. Loading point clouds...
2. Preparing point clouds...
3. Running fast registration...
Trying combination 1/625 (X: -15.0°, Y: -15.0°, Z: -15.0°, offset: -10mm)
New best fitness: 0.251024 (X: -15.0°, Y: -15.0°, Z: -15.0°, offset: -10mm)
Trying combination 2/625 (X: -15.0°, Y: -15.0°, Z: -15.0°, offset: -5mm)
New best fitness: 0.492323 (X: -15.0°, Y: -15.0°, Z: -15.0°, offset: -5mm)
Trying combination 3/625 (X: -15.0°, Y: -15.0°, Z: -15.0°, offset: 0mm)
New best fitness: 0.749744 (X: -15.0°, Y: -15.0°, Z: -15.0°, offset: 0mm)
Trying combination 27/625 (X: -15.0°, Y: -7.5°, Z: -15.0°, offset: -5mm))
New best fitness: 0.751535 (X: -15.0°, Y: -7.5°, Z: -15.0°, offset: -5mm)
Trying combination 32/625 (X: -15.0°, Y: -7.5°, Z: -7.5°, offset: -5mm))
New best fitness: 0.751791 (X: -15.0°, Y: -7.5°, Z: -7.5°, offset: -5mm)
Trying combination 59/625 (X: -15.0°, Y: 0.0°, Z: -7.5°, offset: 5mm))))
New best fitness: 0.783009 (X: -15.0°, Y: 0.0°, Z: -7.5°, offset: 5mm)
Trying combination 323/625 (X: